In [48]:
import configparser
import pandas as pd
import numpy as np

from os.path import join
from itertools import product
from scipy.stats import binom

from solver import Solver

In [9]:
ROOT = ''

SCENARIOS = configparser.ConfigParser()
SCENARIOS.read(join(ROOT, 'scenarios.ini'))
scenario_name = "stay_075__room_1__ind_05"
instance_id = 4

UPGRADE_RULE = 'up'
algo_on = 0
with_capacity_reservation = 0
with_agent_cancel = 0
solution_dir = (f"solution_{UPGRADE_RULE}/algo_{algo_on}/"
                f"overbooking_{with_capacity_reservation}__"
                f"agent_cancel_{with_agent_cancel}")
solution_dir = join(ROOT, solution_dir, scenario_name)

NUM_TYPE = 2
TIME_SPAN_LEN = 6
DEMAND_UB = np.array([20, 10])

In [32]:
g_reservation = pd.read_csv(join(solution_dir, f'{instance_id}_revservation.csv'))
g_valid_res = pd.read_csv(join(solution_dir, f'{instance_id}_ind_valid.csv'))
order_acceptance = pd.read_csv(join(solution_dir, f'{instance_id}_acceptance.csv'),
                               index_col=0).to_numpy().flatten()
upgrade_df = pd.read_csv(join(solution_dir, f'{instance_id}_upgrade.csv'))

In [91]:
g_valid_res
g_valid_np = np.zeros((NUM_TYPE, TIME_SPAN_LEN, 21, 21))
for i, v in g_valid_res.iterrows():
    g_valid_np[int(v['room type']-1), int(v['time']-1), int(v['demand ID']-1),
               int(v['reservation ID']-1)] = v['valid']

In [11]:
g_res_np = np.zeros((NUM_TYPE, TIME_SPAN_LEN, DEMAND_UB.max() + 1))
for i, v in g_reservation.iterrows():
    g_res_np[int(v.room - 1), int(v.time - 1), int(v.demand_ID - 1)] = v.rev

In [17]:
g_upgrade_np = np.zeros((4, NUM_TYPE, NUM_TYPE))
for i, v in upgrade_df.iterrows():
    g_upgrade_np[int(v.order - 1), int(v['from type'] - 1), int(v['to type'] - 1)] = v['upgrade amount']

In [19]:
NUM_ORDER = 4

In [21]:
solver = Solver(
    SCENARIOS[scenario_name],
    instance_id,
    UPGRADE_RULE,
    with_agent_cancel=with_agent_cancel,
    with_capacity_reservation=with_capacity_reservation,
    with_ind_cancel=1,
    data_root='data'
)

In [39]:
upgrade_diff = (
    np.dot(g_upgrade_np, -np.ones((NUM_TYPE, 1))).reshape(
        (NUM_ORDER, NUM_TYPE)
    ) +
    np.dot(np.ones(NUM_TYPE), g_upgrade_np).reshape(
        (NUM_ORDER, NUM_TYPE)
    )
)
acc_order_msk = np.where(order_acceptance == 1)[0]
# order x room
acc_order_room = solver.agent_order_room_quantity + upgrade_diff
acc_order_room = acc_order_room[acc_order_msk]
acc_order_stay = solver.agent_order_stay[acc_order_msk]
# room x time
agent_det_agg_consump = np.dot(
    acc_order_room.T,
    acc_order_stay
)

acc_order_req = acc_order_room[:, :, np.newaxis] * \
    acc_order_stay[: ,np.newaxis, :]
acc_order_cancel_rate = solver.agent_cancel_rate[acc_order_msk]

In [42]:
acc_order_upgrade_profit = (
    g_upgrade_np * solver.upgrade_fee *
    solver.agent_order_stay.sum(axis=1).reshape(solver.num_order, 1, 1)
).sum(axis=(1, 2))[acc_order_msk]
acc_order_price = solver.agent_order_price[acc_order_msk]

In [44]:
ind_rev_id_ub = solver.individual_demand_pmf.shape[2]
if not solver.with_capacity_reservation:
    left_vac = solver.capacity - agent_det_agg_consump
    left_vac[left_vac < 0] = 0
    reservation = np.minimum(
        np.repeat(
            left_vac[:, :, np.newaxis],
            ind_rev_id_ub,
            axis=2
        ),
        np.resize(
            np.arange(ind_rev_id_ub),
            solver.individual_demand_pmf.shape
        )
    )

In [46]:
reservation[0, :, 21:] = 0
reservation[1, :, 11:] = 0

In [123]:
np.allclose(g_res_np, reservation)

True

In [160]:
ind_profit = 0
for room_type, time_id, in product(
    range(solver.num_type), range(solver.time_span_len),
):
    for outcome_id in range(solver.demand_ub[room_type] + 1):
        ind_arrival = np.round(reservation[room_type, time_id, outcome_id])
        ind_arrival_prob = solver.individual_demand_pmf[
            room_type, time_id, outcome_id
        ]
        if (ind_arrival_prob == 0):
            continue  # invalid outcome or no need to count
            # should not run to this block

        # individual profit would be zero without arrival
        # however compensation should count
        # if ind_arrival != 0:
        ind_cancel_probs = binom.pmf(
            np.arange(ind_arrival + 1),
            ind_arrival,
            solver.individual_cancel_rate[room_type]
        )
        ind_real = ind_arrival + np.dot(- np.arange(ind_arrival + 1),
                                        ind_cancel_probs)
        ind_profit += (
            ind_arrival_prob *
            ind_real *
            solver.individual_room_price[room_type]
        )

In [73]:
binom.pmf(
            np.arange(1.9999999999997762 + 1),
            2,
            solver.individual_cancel_rate[0]
        )

array([0.5625, 0.375 , 0.0625])

In [72]:
0.5625 +  0.375

0.9375

In [161]:
ind_profit

54518.352603981606

In [129]:
ind_profit

55641.88195158211

In [162]:
55641.88195158211 - 54518.352603981606

1123.5293476005027

In [67]:
ind_profit + acc_order_price.sum() + acc_order_upgrade_profit.sum()

349796.2852060675

In [68]:
349796.2852060675 - 348672.7559

1123.529306067503

In [56]:
np.allclose(g_res_np, reservation)

True

In [156]:
g_ind_profit = 0
for room_type, t, demand_id in \
    product(np.arange(NUM_TYPE), range(TIME_SPAN_LEN), range(21)):
    for reservation_id in range(demand_id + 1):
        tmp = 0
        for cancel_val in range(reservation_id + 1):
            tmp += binom.pmf(
                cancel_val,
                reservation_id,
                solver.individual_cancel_rate[room_type]
            ) * (reservation_id - cancel_val) * solver.individual_room_price[room_type]
        g_ind_profit += (
            solver.individual_demand_pmf[room_type][t][demand_id] *
            g_valid_np[room_type, t, demand_id, reservation_id] * tmp
        )

In [155]:
valid = np.zeros((NUM_TYPE, TIME_SPAN_LEN, 21, 21))
for room_type, t, demand_id in \
    product(np.arange(NUM_TYPE), range(TIME_SPAN_LEN), range(21)):
        valid[room_type, t, demand_id, int(np.round(reservation[room_type, t, demand_id]))] = 1


In [149]:
np.allclose(valid, g_valid_np)

False

In [153]:
g_valid_np[1, 0, 11]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [154]:
valid[1, 0, 11]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [157]:
g_ind_profit

62136.43573311961

In [144]:
np.where(valid != g_valid_np)

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], dtype=int64),
 array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 11, 12, 13, 14,
        15, 16, 17, 18, 19, 20, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 11,
        12, 13, 14, 15, 16, 17, 18, 19, 20], dtype=int64),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64))

In [126]:
g_ind_profit

54518.352603981606

In [130]:
55641.88195158211 - 54518.352603981606

1123.5293476005027

In [127]:
ind_profit

55146.30662831318

In [122]:
reservation[np.where(g_res_np > reservation)][0]

1.9999999999997762

In [121]:
g_res_np[np.where(g_res_np > reservation)][0]

2.0

In [52]:
349300.7098827986 - 348672.7559


627.9539827986155

In [ ]:
1123.5293476005318